# 5. LightGBM Model 2

Based on this notebook: https://www.kaggle.com/code/ragnar123/amex-lgbm-dart-cv-0-7963

In [ ]:
import gc
import os
import joblib
import random
import warnings
import itertools
import scipy as sp
import numpy as np
import pandas as pd
from tqdm import tqdm
import lightgbm as lgb
from itertools import combinations
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from sklearn.preprocessing import LabelEncoder
import warnings; warnings.filterwarnings('ignore')
from sklearn.model_selection import StratifiedKFold, train_test_split

## Preprocessing

In [ ]:
def get_difference(data, num_features):
    df1,df3,df6 = [],[],[]
    customer_ids = []
    for customer_id, df in tqdm(data.groupby(['customer_ID'])):
        diff_df1 = df[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
        diff_df3 = df[num_features].diff(3).iloc[[-1]].values.astype(np.float32)
        diff_df6 = df[num_features].diff(6).iloc[[-1]].values.astype(np.float32)

        df1.append(diff_df1)
        df3.append(diff_df3)
        df6.append(diff_df6)
        customer_ids.append(customer_id)
        
    df1 = np.concatenate(df1, axis = 0)
    df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_features].columns])
    df3 = np.concatenate(df3, axis = 0)
    df3 = pd.DataFrame(df3, columns = [col + '_diff3' for col in df[num_features].columns])
    df6 = np.concatenate(df6, axis = 0)
    df6 = pd.DataFrame(df6, columns = [col + '_diff6' for col in df[num_features].columns])
    df_all = pd.concat([df1,df3,df6],axis=1)
    df_all['customer_ID'] = customer_ids
    return df_all

def read_preprocess_data():
    train = pd.read_parquet('../input/amex-data-integer-dtypes-parquet-format/train.parquet')
    features = train.drop(['customer_ID', 'S_2'], axis = 1).columns.to_list()
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68",
    ]
    num_features = [col for col in features if col not in cat_features]
    print('Starting training feature engineer...')
    train_num_agg = train.groupby("customer_ID")[num_features].agg(['first', 'mean', 'std', 'min', 'max'])
    train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
    train_num_agg.reset_index(inplace = True)
    train_tail2 = train.groupby("customer_ID").tail(2)
    train_tail2_num_agg = train_tail2.groupby("customer_ID")[num_features].agg(['mean'])
    train_tail2_num_agg.columns = ['_'.join([xx.replace('mean','last') for xx in x]) for x in train_tail2_num_agg.columns]
    train_tail2_num_agg.reset_index(inplace = True)
    train_num_agg = train_num_agg.merge(train_tail2_num_agg, how = 'inner', on = 'customer_ID')
    for col in num_features:
        train_num_agg[f'{col}_last_mean_diff'] = train_num_agg[f'{col}_last'] - train_num_agg[f'{col}_mean']
        train_num_agg[f'{col}_last_first_diff'] = train_num_agg[f'{col}_last'] - train_num_agg[f'{col}_first']
    train_cat_agg = train.groupby("customer_ID")[cat_features].agg(['count', 'first', 'last', 'nunique'])
    train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
    train_cat_agg.reset_index(inplace = True)
    train_labels = pd.read_csv('../input/amex-default-prediction/train_labels.csv')
    cols = list(train_num_agg.dtypes[train_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        train_num_agg[col] = train_num_agg[col].astype(np.float32)
    cols = list(train_cat_agg.dtypes[train_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        train_cat_agg[col] = train_cat_agg[col].astype(np.int32)
    train_diff = get_difference(train, num_features)
    train = train_num_agg.merge(train_cat_agg, how = 'inner', on = 'customer_ID').\
            merge(train_diff, how = 'inner', on = 'customer_ID').\
            merge(train_labels, how = 'inner', on = 'customer_ID')
    del train_num_agg, train_cat_agg, train_diff, train_tail2_num_agg
    gc.collect()
    
    test = pd.read_parquet('../input/amex-data-integer-dtypes-parquet-format/test.parquet')
    print('Starting test feature engineer...')
    test_num_agg = test.groupby("customer_ID")[num_features].agg(['first', 'mean', 'std', 'min', 'max'])
    test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]
    test_num_agg.reset_index(inplace = True)
    test_tail2 = test.groupby("customer_ID").tail(2)
    test_tail2_num_agg = test_tail2.groupby("customer_ID")[num_features].agg(['mean'])
    test_tail2_num_agg.columns = ['_'.join([xx.replace('mean','last') for xx in x]) for x in test_tail2_num_agg.columns]
    test_tail2_num_agg.reset_index(inplace = True)
    test_num_agg = test_num_agg.merge(test_tail2_num_agg, how = 'inner', on = 'customer_ID')
    for col in num_features:
        test_num_agg[f'{col}_last_mean_diff'] = test_num_agg[f'{col}_last'] - test_num_agg[f'{col}_mean']
        test_num_agg[f'{col}_last_first_diff'] = test_num_agg[f'{col}_last'] - test_num_agg[f'{col}_first']
    test_cat_agg = test.groupby("customer_ID")[cat_features].agg(['count', 'first', 'last', 'nunique'])
    test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]
    test_cat_agg.reset_index(inplace = True)
    cols = list(test_num_agg.dtypes[test_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        test_num_agg[col] = test_num_agg[col].astype(np.float32)
    cols = list(test_cat_agg.dtypes[test_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        test_cat_agg[col] = test_cat_agg[col].astype(np.int32)
    test_diff = get_difference(test, num_features)
    test = test_num_agg.merge(test_cat_agg, how = 'inner', on = 'customer_ID').\
            merge(test_diff, how = 'inner', on = 'customer_ID')
    del test_num_agg, test_cat_agg, test_diff
    gc.collect()

    features = train.drop(['customer_ID'], axis = 1).columns.to_list()
    num_features = [col for col in features if col not in cat_features]
    num_cols = [col for col in num_features if (('last' in col or 'mean' in col) and 'diff' not in col)]
    for col in num_cols:
        train[col + '_round2'] = train[col].round(2)
        test[col + '_round2'] = test[col].round(2)
    print('train.shape:',train.shape)
    print('test.shape:',test.shape)

    train.to_parquet('train_fe.parquet')
    test.to_parquet('test_fe.parquet')

In [ ]:
read_preprocess_data()

## Training and Inference

In [ ]:
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations
from catboost import CatBoostClassifier

In [ ]:
# ====================================================
# Configurations
# ====================================================
class CFG:
    input_dir = './'
    seed = 42
    n_folds = 5
    target = 'target'
    boosting_type = 'dart'
    metric = 'binary_logloss'

# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# ====================================================
# Read data
# ====================================================
def read_data():
    train = pd.read_parquet(CFG.input_dir + 'train_fe_v2.parquet')
    test = pd.read_parquet(CFG.input_dir + 'train_fe_v2.parquet')
    return train, test

# ====================================================
# Amex metric
# ====================================================
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

# ====================================================
# LGBM amex metric
# ====================================================
def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred), True

In [ ]:
# ====================================================
# Train & Evaluate
# ====================================================
def train_and_evaluate(train, test):
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68"
    ]
    cat_features = [f"{cf}_last" for cf in cat_features] +  [f"{cf}_first" for cf in cat_features]
    for cat_col in cat_features:
        encoder = LabelEncoder()
        train[cat_col] = encoder.fit_transform(train[cat_col])
        test[cat_col] = encoder.transform(test[cat_col])
    num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
    features = [col for col in train.columns if col not in ['customer_ID', CFG.target]]
    features = [col for col in features if 'B_29' not in col]
    params = {
        'objective': 'binary',
        'metric': CFG.metric,
        'boosting': CFG.boosting_type,
        'seed': CFG.seed,
        'num_leaves': 100,
        'learning_rate': 0.01,
        'feature_fraction': 0.20,
        'bagging_freq': 10,
        'bagging_fraction': 0.50,
        'n_jobs': -1,
        'lambda_l2': 2,
        'min_data_in_leaf': 40,
        }
    test_predictions = np.zeros(len(test))
    oof_predictions = np.zeros(len(train))
    kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):
        print(' ')
        print('-'*50)
        print(f'Training fold {fold} with {len(features)} features...')
        x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
        y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]
        model = CatBoostClassifier(iterations=10000,learning_rate=0.03, random_state=22,task_type='GPU')
        model.fit(x_train, y_train, eval_set=[(x_val, y_val)], cat_features=cat_features,  verbose=100, use_best_model=True)
        joblib.dump(model, f'cat_fold{fold}_seed{CFG.seed}.pkl')
        val_pred = model.predict_proba(x_val)[:, 1]
        oof_predictions[val_ind] = val_pred
        test_pred_list = []
        for ii in tqdm(range(int(len(test)/10000)+1)):
            test_pred_tmp = model.predict_proba(test[ii*10000:(ii+1)*10000][features])[:, 1]
            test_pred_list.append(test_pred_tmp)
        test_pred = np.concatenate(test_pred_list)
        test_predictions += test_pred / CFG.n_folds
        score = amex_metric(y_val, val_pred)
        print(f'Our fold {fold} CV score is {score}')
        del x_train, x_val, y_train, y_val
        gc.collect()
    score = amex_metric(train[CFG.target], oof_predictions)
    print(f'Our out of folds CV score is {score}')
    oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[CFG.target], 'prediction': oof_predictions})
    oof_df.to_csv(f'oof_cat_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)
    test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
    test_df.to_csv('../sub/test_lgbm_5fold_seed42_v2.csv', index = False)

In [ ]:
seed_everything(CFG.seed)
train, test = read_data()
print(train.shape)

In [ ]:
train_and_evaluate(train, test)